# Bayesian AB Testing Part 3 - Picking a Winner

In [2]:
library(dplyr)
library(rjags)

In [74]:
# Create some fake data 
set.seed(2021)

n = 120
p_a = 0.29
p_b = 0.5

x_a = sum(rbinom(n = n, size = 1, prob = p_a))
x_b = sum(rbinom(n = n, size = 1, prob = p_b))

In [ ]:
model = "model {
  
  for(i in 1:2){

    # priors
    logits[i] ~ dnorm(0, 1)
  
    # Transform from logit to success prob.
    theta[i] <- 1.0/(1.0 + exp(-logits[i]))
    
    x[i] ~ dbin(theta[i], N)
  } 

    diff <- theta[1] - theta[2]

    # probability to be best
    ptbb[1] <- step(theta[1] - theta[2]) # will be 1 if > 0 and 0 otherwise
    ptbb[2] <- step(theta[2] - theta[1])
    
    # Expected loss 
    loss[1] <- step(theta[2] - theta[1])*(theta[2] - theta[1]) 
    loss[2] <- step(theta[1] - theta[2])*(theta[1] - theta[2])
    
}
"
jm = jags.model(textConnection(model), data = list(N = n, x = c(x_a, x_b)))
out = coda.samples(jm, variable.names = c("theta", 'diff', 'ptbb', 'loss'), n.iter = 100000, thin=10)
results = as.data.frame(out[[1]])

In [ ]:
source('functions/decision-functions.R')

In [ ]:
plot_loss(ps$loss_theta_a, ps$loss_theta_b)

In [ ]:
rope(ps$theta_a, ps$theta_b)